## Install / Import

In [1]:
#!pip install face_recognition

In [1]:
import face_recognition
import cv2
import matplotlib.pyplot as plt
import time
import numpy as np
import pickle
#from google.colab.patches import cv2_imshow

## Load encodings

In [2]:
pickle_name = "brain.pickle"
data_encoding = pickle.loads(open(pickle_name, "rb").read())
list_encodings = data_encoding["encodings"]
list_names = data_encoding["names"]

## Helper functions

In [3]:
max_width = 900

In [4]:
def resize_video(width, height, max_width = 600):
  if (width > max_width):
    proportion = width / height
    video_width = max_width
    video_height = int(video_width / proportion)
  else:
    video_width = width
    video_height = height

  return video_width, video_height

In [5]:
resizing = 0.5

In [6]:
def recognize_faces(image, list_encodings, list_names, resizing=0.25, tolerance=0.6):
  image = cv2.resize(image, (0, 0), fx=resizing, fy=resizing)

  img_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
  face_locations = face_recognition.face_locations(img_rgb)
  face_encodings = face_recognition.face_encodings(img_rgb, face_locations)

  face_names = []
  conf_values = []
  for encoding in face_encodings:
    matches = face_recognition.compare_faces(list_encodings, encoding, tolerance=tolerance)
    name = "Not identified"

    face_distances = face_recognition.face_distance(list_encodings, encoding)
    best_match_index = np.argmin(face_distances)
    if matches[best_match_index]:
      name = list_names[best_match_index]
    face_names.append(name)
    conf_values.append(face_distances[best_match_index])

  face_locations = np.array(face_locations)
  # we are scalig back up the face locations, since the frame we detected was scaled by the scaleFactor of `resizing` variable
  face_locations = face_locations / resizing
  return face_locations.astype(int), face_names, conf_values

In [7]:
def show_recognition(frame, face_locations, face_names, conf_values):

  for face_loc, name, conf in zip(face_locations, face_names, conf_values):
    y1, x2, y2, x1 = face_loc[0], face_loc[1], face_loc[2], face_loc[3]

    conf = "{:.8f}".format(conf)
    cv2.putText(frame, name,(x1, y1 - 10), cv2.FONT_HERSHEY_DUPLEX, 0.7, (20, 255, 0), 2, lineType=cv2.LINE_AA)
    cv2.rectangle(frame, (x1, y1), (x2, y2), (20, 255, 0), 4)
    if name != "Not identified":
        cv2.putText(frame, conf,(x1, y2 + 15), cv2.FONT_HERSHEY_DUPLEX, 0.5, (20, 255, 0), 1, lineType=cv2.LINE_AA)

  return frame

## Video Configurations

In [8]:
fourcc = cv2.VideoWriter_fourcc(*'XVID')
fps = 30

## Reading the video file

In [9]:
video_file = r".\data\cropped_images\test\Video.mp4"
result_file = 'result.avi'

cap = cv2.VideoCapture(video_file)
connected, video = cap.read()
video_width, video_height = video.shape[1], video.shape[0]

if max_width is not None:
  video_width, video_height = resize_video(video_width, video_height, max_width)
print(video_width, video_height)

video_output = cv2.VideoWriter(result_file, fourcc, fps, (video_width, video_height))

402 850


## Processing the video

In [10]:
frames_show = 24
current_frame = 1
max_frames = -1

## Face Recognition in Videos  

In [11]:
while (cv2.waitKey(1) < 0):
  connected, frame = cap.read() 

  if not connected:
    break  
  
  if max_frames > -1 and current_frame > max_frames:
      break   
  
  (H, W) = frame.shape[:2] 
  
  t = time.time() 

  if max_width is not None: 
    frame = cv2.resize(frame, (video_width, video_height)) 

  face_locations, face_names, conf_values = recognize_faces(frame, list_encodings, list_names, resizing)
  frame = show_recognition(frame, face_locations, face_names, conf_values)
                          
  cv2.putText(frame, " frame processed in {:.2f} seconds".format(time.time() - t), (20, video_height-20), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (250, 250, 250), 0, lineType=cv2.LINE_AA)  # cv2.LINE_AA to improve the quality of the text

  video_output.write(frame) 

  if current_frame <= frames_show:
    cv2.imshow('Resized Image', cv2.resize(frame, (0, 0), fx=0.75, fy=0.75))

  current_frame + 1 

print("Finished")
video_output.release() 
cv2.destroyAllWindows()